In [1]:
import pandas as pd
import numpy as npa
import math
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import cohen_kappa_score
warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',None)

In [2]:
abhishek_results = pd.read_csv('130_results_abhishek.csv').to_dict('records')
david_results = pd.read_csv('130_results_david.csv').to_dict('records') 

In [3]:
abhishek_results_map = {element['commit_hash']: element for element in abhishek_results }
david_results_map = {element['commit_hash']: element for element in david_results }

In [4]:
def sameResult(r1,r2):
    if r1 == r2: return True
    if r1 == 'none': 
        if r2 == 'unknown': return True
    if r1 == 'unknown': 
        if r2 == 'none': return True
    return False

In [5]:
categories = {
    "Timing and execution": 0,
    "Exchange of Information": 1,
    "Memory": 2,
    "unknown": 3
}

In [6]:
same_is_bug_fixing_commit = 0
is_bug_fixing_commit_conflict = []
both_is_bug_fixing_commit = 0

same_is_safety_related = 0
is_safety_related_conflict = []
both_is_safety_related = 0

same_type_of_safety_related = 0
type_of_safety_related_conflict = []

kappa_cohen_results = {
    'is_bug_fixing_commit': { 'david': [], 'Abhishek': [] },
    'is_safety_related': { 'david': [], 'Abhishek': [] },
    'type_of_safety_related': { 'david': [], 'Abhishek': [] }
}

for d_result in david_results:
    a_result = abhishek_results_map[d_result['commit_hash']]

    kappa_cohen_results['is_bug_fixing_commit']['david'].append(1 if d_result['is_bug_fixing_commit'] else 0)
    kappa_cohen_results['is_bug_fixing_commit']['Abhishek'].append(1 if a_result['is_bug_fixing_commit'] else 0)
    
    # IS BUG-FIXING COMMIT
    if a_result['is_bug_fixing_commit'] == d_result['is_bug_fixing_commit']:
        same_is_bug_fixing_commit += 1
    else:
        is_bug_fixing_commit_conflict.append(
            (a_result['is_bug_fixing_commit'],a_result['comment'],d_result['is_bug_fixing_commit'], d_result['comment'])
        )

    # CASES WHERE BOTH AGREE ON "IS BUG-FIXING COMMIT"
    if a_result['is_bug_fixing_commit'] and d_result['is_bug_fixing_commit']:

        both_is_bug_fixing_commit+=1
        
        kappa_cohen_results['is_safety_related']['david'].append(
            1 if d_result['is_safety_related'] == 'true' else 0 if d_result['is_safety_related'] == 'false' else 2
        )
        kappa_cohen_results['is_safety_related']['Abhishek'].append(
            1 if a_result['is_safety_related'] == 'true' else 0 if a_result['is_safety_related'] == 'false' else 2
        )  
        
        # IS SAFETY-RELATED BUG
        if a_result['is_safety_related'] == d_result['is_safety_related']:
            same_is_safety_related += 1

            # CASES WHERE BOTH AGREE ON "SAFETY-RELATED BUG"
            if a_result['is_safety_related'] == 'true' and d_result['is_safety_related'] == 'true':
                both_is_safety_related += 1
                
                kappa_cohen_results['type_of_safety_related']['david'].append(categories[d_result['type_of_safety_related']])
                kappa_cohen_results['type_of_safety_related']['Abhishek'].append(categories[a_result['type_of_safety_related']])
                
                # SAFETY-RELATED BUG CATEGORY
                if a_result['type_of_safety_related'] == d_result['type_of_safety_related']:
                    same_type_of_safety_related += 1
                else:
                    type_of_safety_related_conflict.append(
                        (a_result['type_of_safety_related'], a_result['comment'], d_result['type_of_safety_related'], d_result['comment'])
                    )
        
        else:
            is_safety_related_conflict.append(
                (a_result['is_safety_related'], a_result['comment'], d_result['is_safety_related'], d_result['comment'])
            )

print("same_is_bug_fixing_commit: ", same_is_bug_fixing_commit, "/", len(david_results), "(%.2f)"%(same_is_bug_fixing_commit*100/len(david_results)))
print("same_is_safety_related: ", same_is_safety_related, "/", both_is_bug_fixing_commit, "(%.2f)"%(same_is_safety_related*100/both_is_bug_fixing_commit))
print("same_type_of_safety_related: ", same_type_of_safety_related, "/", both_is_safety_related, "(%.2f)"%(same_type_of_safety_related*100/both_is_safety_related))

same_is_bug_fixing_commit:  109 / 130 (83.85)
same_is_safety_related:  20 / 49 (40.82)
same_type_of_safety_related:  8 / 13 (61.54)


In [7]:
cohen_kappa_score(kappa_cohen_results['is_bug_fixing_commit']['david'], kappa_cohen_results['is_bug_fixing_commit']['Abhishek'])

0.6801031169439888

In [8]:
cohen_kappa_score(kappa_cohen_results['is_safety_related']['david'], kappa_cohen_results['is_safety_related']['Abhishek'])

0.1382656155245604

In [9]:
cohen_kappa_score(kappa_cohen_results['type_of_safety_related']['david'], kappa_cohen_results['type_of_safety_related']['Abhishek'])

0.45378151260504196

### Not same "Is a bug-fixing commit" reasoning

In [10]:
pd.DataFrame(is_bug_fixing_commit_conflict, columns =['Abhishek result', 'Abhishek comment', 'david result', 'david comment'])

,Abhishek result,Abhishek comment,david result,david comment
0,True,NaN,False,Looks like an update more than a bug fixing
1,True,NaN,False,Looks like a feature more than a fix.
2,True,NaN,False,It seems that they are refactoring more than fixing.
3,True,NaN,False,Moving a return; nothing related to a bug fixing.
4,False,NaN,True,They are fixing with this pull a livepatch bug; could be considered as safety related commit because it fixes an init fail of Livepatch.
5,True,NaN,False,Refactoring variable name.
6,True,NaN,False,Refactoring for a better future debug.
7,True,NaN,False,It is a perfomance feature; in my opinion (I think).
8,True,NaN,False,Looks like a feature.
9,True,NaN,False,Removing a false-positive warning.


### Not same "Is a safety-related bug" reasoning

In [11]:
pd.DataFrame(is_safety_related_conflict, columns =['Abhishek result', 'Abhishek comment', 'david result', 'david comment'])

,Abhishek result,Abhishek comment,david result,david comment
0,true,Also I think it falls under the exchange of information category because the kernel-infoleak fix relates to loss or leak of information.,unknown,It is a fix (merge commit); as the message says it.
1,true,NaN,unknown,It is a fix; as the message says it; fixing a bug on the driver when detecting HBM.
2,false,NaN,unknown,Seems a fix of a bug; but don't know what kind of bug.
3,false,NaN,unknown,I would say that this commit is fixing a bug about a GPU error when initialization; maybe is considered as safety related.
4,true,Not 100% sure about the safety-type,unknown,I think they are fixing several bugs here.
5,true,NaN,unknown,It is fixing a bug; but I don't know if it safety related.
6,false,NaN,unknown,They are fixing with this pull a livepatch bug; could be considered as safety related commit because it fixes an init fail of Livepatch.
7,false,NaN,true,Fixing an bug that causes an exit error; considered as safety related bug.
8,false,NaN,unknown,It is fixing a but; but I don't understand the type.
9,false,NaN,unknown,They are fixing several bugs here; but I don't know if they are related to safety.


### Not same "Safety-related bug category" reasoning

In [12]:
pd.DataFrame(type_of_safety_related_conflict, columns =['Abhishek result', 'Abhishek comment', 'david result', 'david comment'])

,Abhishek result,Abhishek comment,david result,david comment
0,Exchange of Information,NaN,Timing and execution,It is fixing a concurrency bug; so it is safety related.
1,Exchange of Information,NaN,Memory,In this commit; they talk about fixing; and inside the message they specify some changes related to memory.
2,Timing and execution,NaN,unknown,Fixing a bug when about MPC that hang the system.
3,Memory,NaN,Timing and execution,They talk about a bug fixed; this bg was related to concurrency; so it is related to time and execution as they are talking about locks as well.
4,Exchange of Information,Not 100% sure,Timing and execution,It is fixing commit of a driver that hangs in an specific situation.
